## Steps

This notebook shows how to use Amazon Personalize's new user personalization recipe (aws-user-personalization). This recipe balances recommendations between new and old items, allowing you to adjust the balance in favor of more new or more old items

At a high level, using the new USER Personalization recipe involves the below steps:  

1. Setup Personalize client
2. Create DatasetGroup, Define a Schema, Import Datasets and Ingest real-time Interactions
4. Create a Campaign with a new config `campaignConfig`
5. Create a Event Tracker to ingest the events sent by PutEvents.
6. Call GetRecommendations, a new field `RecommendationId` is returned in the response.
7. Call putEvents with `RecommendationId` or a custom list of Impression Items.
8. Wait for the campaign to be updated.
9. Update campaign to stop the auto update.
10. Cleanup


> **NOTE:**: **Execution of this notebook will take a couple of hours.**

### Access Key/Secret Key setup for AWS API access.

Make sure the accessKey, secretKey you use have the approriate permissions. Also choose the region you want to run this demo

In [ ]:
accessKeyId = ""
secretAccessKey = ""
region_name = ""

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

### 1. Client setup
Let's first setup the client for personalize and s3.

In [ ]:
# Public s3 bucket owned by Personalize service which used to store the example dataset.

personalize_s3_bucket = "personalize-cli-json-models"
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')



####  Initialize personalize clients

In [ ]:

personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### Sample Datasets

For convenience and for the purposes of this demo, we'll use the sample datasets provided by personalize.
We provided two dataset, one is item metadata, another one is interaction dataset. Let's first download it to local

In [ ]:
interaction_dataset_key = "sample-dataset/interactions-sample.csv"
items_dataset_key = "sample-dataset/items-with-creation-timestamp-sample.csv"
interactions_file = os.getcwd() + "/interaction_raw.csv"
items_metadata_file = os.getcwd() + "/items_raw.csv"
s3_client.download_file(personalize_s3_bucket, interaction_dataset_key, interactions_file)
s3_client.download_file(personalize_s3_bucket, items_dataset_key, items_metadata_file)

In [ ]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### Let's have a glance at the interaction dataframe

In [ ]:
interactions_df.head(2)

#### Interactions dataset
**ITEM_ID**: Item corresponding to the EVENT_TYPE.

**EVENT_TYPE**: Event type.  

**TIMESTAMP**: Timestamp of the Interaction in milliseconds. (Note that it is in milliseconds) 

**USER_ID**: User Id corresponding to this impression.  

**IMPRESSION:**  Now you could optionally pass impression data along with the event data in the interaction dataset. This is passed in a new field `IMPRESSION`as you can see above which takes a piped concatination of the items that the user interacted with (for example the items that were shown to the user), Impressions also include the clicked Items.  


In [ ]:
items_df.head(2)

#### Items dataset

Items dataset contains Item_ids and associated metadata.

**ITEM_ID**: Item Id  

**genres** Metadata of the Item, if multiple categorical classification for the same item then use a pipe " | " to concatenate.

**creation_timestamp** Timestamp when the item was added

More details about datasets can be found in the documentation
https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

#### Update the timestamp

Here we are going to update the stamp of our dataset to 8 days from today in order to show the impact of real-time interactions on our recommendations

In [ ]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [ ]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### After update, let's check the interaction and item dataset one more time

In [ ]:
interactions_df.head(2)

In [ ]:
items_df.head(2)


### 2. Ingest data to Amazon Personalize
Now lets create a datasetGroup, create Schema, Upload the datasets, create a datasetImport Job. This has not changed.

#### a. Create DatasetGroup

This is similar to the existing recipe, the full documentation can be found [here](https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html)

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [ ]:

print('dataset_group_arn : {}'.format(dataset_group_arn))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### b. Create Dataset Schemas

> **_NOTE:_**: `Impression` field has a type String and uses a piped concatination for multiple values.

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [ ]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
# print(json.dumps(create_schema_response, indent=2))
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

In [ ]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRES",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

# print(json.dumps(create_schema_response, indent=2))
item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

#### c. create Datasets

In [ ]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

#### d. Upload datasets to the S3 bucket, setup approriate S3 Bucket policy, IAM Role, etc.,

We need to upload these datasets or you could provide the bucket name where you already have the datasets

In [ ]:
#!aws s3 mb s3://{s3_bucket_name}
s3_bucket_name

In [ ]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': region_name})

In [ ]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [ ]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. Attach policy to your S3 bucket

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. Setup Approriate IAM Role so Personalize can access the datasets

In [ ]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [ ]:
print('role_arn:', role_arn)

#### g. Create DatasetImportJobs to upload data

In [ ]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

In [ ]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


### 3. Create Solution, SolutionVersion

We will create a solution with 'aws-user-personalization'. This recipe balances recommendations for new and old items delivered to users

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

#### Create SolutionVersion

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 4. Create a campaign
When creating the campaign, we can set the itemExplorationConfig to configure cold-items exploration weight and also exploration age cut-off. For now, we can set higher explorationWeight as 0.9 and explorationItemAgeCutOff to 7, so we think all the item creation time less then 7 days would be considered as cold item, we would do more exploration on those new items.

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

### 5. Call GetRecommendations
For the purposes of demo, we'll use the Userids in the input dataset to make getRecommendation calls. 
> **_NOTE:_**: In the response, you have a new field `RecommendationId` which correspond to the list of Items returned by Personalize GetRecommendations. You can pass this RecommendationId to indicate the Impressions.  
You could also pass Impression as a piped string concatination of items, if you pass both RecommendationId and ImpressionList, ImpressionList would take precedence and used in the system.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')
print(rec_response['recommendationId'])

In [ ]:
rec_response['itemList']

### 6. Create Event Tracker

Creates an event tracker that you use when sending event data to the specified dataset group using the PutEvents API.

In [ ]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

### 7. Send Impression data to Personalize via PutEvents.
Amazon Personalize can model two types of impressions:  
1. Implicit impressions and explicit impressions. Implicit impressions are impressions that occur during a user's session, and are automatically recorded by Amazon Personalize whenever the user is shown an item. You can integrate them into your recommendation workflow by including the RecommendationId (returned by the and operations) as input for future PutEvents requests.  


2. Explicit impressions are impressions that you manually input when making a PutEvents request. You would use explicit Impressions when you for example not show some of the items returned by GetRecommendations due to unavailablity, etc.,   

> **NOTE:** If you have defined `impression` in your Interaction Schema as above, you need to send the impression list(either the items returned from GetRecommendations or your own).  
**When both recommendationId and Impressions are , Amazon Personalize will use the explicit impressions by default.**



#### Let's put the previously recommmended item as impressions

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

#### We can also put some new items

Let's put new itemId '2xx' to the personalize.

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '240',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '241',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '242',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '243',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '244',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '245',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '246',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '247',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '248',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '249',
     }]
    )

### 8. Create new SolutionVersion with updateMode

After put-events, Please wait for around 15 minutes for Personalize to ingest the new data, after that, create new solutionVersion with update-mode

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 9. Update Campaign

Update the campaign with the latest solution-version arn from update.

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

In [ ]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

### After updated the solution version, let's do recommendation again

We would expect new items show in the recommendation, since we set high exploration as 0.9

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response['itemList']

### Update campaign with different explorationWeight

We would expect more old items show in the recommendation list since we set a low explorationWeight.

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [ ]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

### After updated explorationWeight

Let's do recommendation again, we should see more old item here.

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response

#### 10. Delete Resources

After created all resouces, let's cleanup all resources.

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)